# The Battle of Neighborhoods : A location recommendation for opening up a new Vegan Restaurant in Bengaluru city

The dataset used for this project is from a wiki page that contains all neighboorhood names, which I have extracted with the help of Web Scraping using BeautifulSoup4 package. The below one is the link for the data source:

https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore

The business idea is to look for a profitable location to open up a new Vegan Restaurant in Bangalore city , India. 

In [17]:
#neccessary updates
#!pip install -U numpy
#!pip install -U pandas
#!pip install -U scipy
#!pip install -U scikit-learn
#!pip install -U imbalanced-learn

The required modules and libraries are imported.

In [2]:
import numpy as np


import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup # this module helps in web scrapping.


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!pip install geocoder
import geocoder

import requests  # this module helps us to download a web page

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium=0.5.0 --yes 
import folium

# import k-means from clustering stage
from sklearn.cluster import KMeans



print("Successfully imported libraries!")

Successfully imported libraries!


## 1. Data Collection using Web Scraping

In [3]:
#url = "https://www.indiatvnews.com/pincode/karnataka/bangalore/"
url = "https://en.wikipedia.org/wiki/Category:Neighbourhoods_in_Bangalore"
data = requests.get(url).text

In [4]:
#create a soup object of the class BeautifulSoup

soup = BeautifulSoup(data, 'html5lib')


In [5]:
# create a list to store neighborhood data
neighborhoodList = []

In [6]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [7]:
# create a new DataFrame from the list
df_blr = pd.DataFrame({"Neighborhood": neighborhoodList})
df_blr.head(10)


,Neighborhood
0,List of areas in Bangalore Cantonment
1,List of areas in Bengaluru Pete
2,List of neighbourhoods in Bangalore
3,Adugodi
4,"Agara, Bangalore"
5,Ananthnagar
6,Anjanapura
7,Arekere
8,Austin Town
9,Babusapalya


In [8]:
#cleaning the data by removing top 3 rows as they are not supposed to be our dataset's part and then resetting the index, a fresh data is obtained
df_blr.drop(df_blr.index[:3],inplace=True)
df_blr=df_blr.reset_index()
del df_blr['index']

#### The Dataframe that is obtained after cleaning

In [9]:
df_blr.head(10)

,Neighborhood
0,Adugodi
1,"Agara, Bangalore"
2,Ananthnagar
3,Anjanapura
4,Arekere
5,Austin Town
6,Babusapalya
7,"Bagalur, Bangalore Urban"
8,Bahubalinagar
9,Baiyyappanahalli


In [10]:
#the total neighborhoods 
print("The total number of neighborhoods in Bangalore is {}".format(df_blr.shape[0]))

The total number of neighborhoods in Bangalore is 140


#### Gathering coordinates for each neighboorhoods in the data frame.

In [11]:
#function to get the coordinates for all neighborhood in df_blr

def get_cord(neighborhood):
    lat_lon_cord = None
    #looping until we get coordinates
    while(lat_lon_cord is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lon_cord = g.latlng
    return lat_lon_cord

In [12]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_cord(neighborhood) for neighborhood in df_blr["Neighborhood"].tolist() ]

In [41]:
coords

[[12.944010000000048, 77.60797000000008],
 [12.84292000000005, 77.48582000000005],
 [12.954080000000033, 77.54135000000008],
 [12.858110000000067, 77.55910000000006],
 [12.885670000000061, 77.59673000000004],
 [12.963480000000061, 77.61297000000008],
 [13.027180000000044, 77.65045000000003],
 [13.077263629807632, 77.65758952294048],
 [13.044710000000066, 77.55008000000004],
 [12.98929000000004, 77.65111000000007],
 [12.98929000000004, 77.65111000000007],
 [12.922310000000039, 77.56988000000007],
 [13.019526511351998, 77.65502797845224],
 [12.938980000000072, 77.57137000000006],
 [12.986740000000054, 77.54389000000003],
 [12.927340000000072, 77.67169000000007],
 [12.966180000000065, 77.58690000000007],
 [13.083010000000058, 77.54779000000008],
 [12.90008000000006, 77.60432000000003],
 [13.015990000000045, 77.61046000000005],
 [12.983991999946278, 77.6404896228702],
 [12.90307000000007, 77.62445000000008],
 [12.817510000000027, 77.67881000000006],
 [12.993330000000071, 77.66123000000005]

In [13]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [14]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_cord(neighborhood) for neighborhood in df_blr["Neighborhood"].tolist() ]

In [15]:
# merge the coordinates into the original dataframe
df_blr['Latitude'] = df_coords['Latitude']
df_blr['Longitude'] = df_coords['Longitude']

#### The final data frame after merging the coordinates columns with main data frame

In [17]:
# verify the neighborhoods and the coordinates in the dataframe
print(df_blr.shape)
df_blr.head(10)

(140, 3)


,Neighborhood,Latitude,Longitude
0,Adugodi,12.944010,77.60797
1,"Agara, Bangalore",12.842920,77.48582
2,Ananthnagar,12.954080,77.54135
3,Anjanapura,12.858110,77.55910
4,Arekere,12.885670,77.59673
5,Austin Town,12.963480,77.61297
6,Babusapalya,13.027180,77.65045
7,"Bagalur, Bangalore Urban",13.077264,77.65759
8,Bahubalinagar,13.044710,77.55008
9,Baiyyappanahalli,12.989290,77.65111


## 2. Plotting the city's map using folium and with coordinates of each neighborhood

#### First, we get the coordinates of Bengaluru and then create the map using folium 

In [18]:
# get the coordinates of Bangalore
address = 'Bangalore, India'

geolocator = Nominatim(user_agent="blr-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bangalore, India 12.9791198, 77.5912997.


In [19]:
# create map of Hyderabad using latitude and longitude values
blr_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(df_blr['Latitude'], df_blr['Longitude'], df_blr['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(blr_map)  
    
blr_map

## 3. Explore the neighborhoods using Foursquare API

Define the client credentials and version

In [20]:
# The code was removed by Watson Studio for sharing.

In [21]:
#define the radius and limits of venue
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(df_blr['Latitude'], df_blr['Longitude'], df_blr['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [23]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head(10)

(7523, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Adugodi,12.94401,77.60797,Zingron - Naga Kitchen,12.936271,77.615051,Indian Restaurant
1,Adugodi,12.94401,77.60797,Flight 4 Fantasy,12.934877,77.611913,Gaming Cafe
2,Adugodi,12.94401,77.60797,The Chocolate Room,12.935505,77.608689,Café
3,Adugodi,12.94401,77.60797,PVR IMAX,12.934595,77.611321,Movie Theater
4,Adugodi,12.94401,77.60797,Dyu Art Cafe,12.937289,77.617591,Financial or Legal Service
5,Adugodi,12.94401,77.60797,Sree Krishna Kafe,12.937064,77.619729,Breakfast Spot
6,Adugodi,12.94401,77.60797,Truffles Ice & Spice,12.933443,77.614265,Burger Joint
7,Adugodi,12.94401,77.60797,Cafe Aloha,12.935651,77.608078,Café
8,Adugodi,12.94401,77.60797,Bliss Chocolate Lounge,12.934123,77.610974,Dessert Shop
9,Adugodi,12.94401,77.60797,Gilly's Rest-O-Bar,12.932987,77.614755,Bar


## 4. Venues per neighborhood

In [24]:
# Let's see how many venues were returned per neighborhood
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Ananthnagar,26,26,26,26,26,26
Anjanapura,5,5,5,5,5,5
Arekere,82,82,82,82,82,82
Austin Town,100,100,100,100,100,100
BTM Layout,91,91,91,91,91,91
Babusapalya,54,54,54,54,54,54
"Bagalur, Bangalore Urban",12,12,12,12,12,12
Bahubalinagar,21,21,21,21,21,21


## 5. Number of unique categories that can be curated from all returned venues

In [25]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 243 uniques categories.


In [26]:
venues_df['VenueCategory'].unique()[:50] #displays all the 237 category names

array(['Indian Restaurant', 'Gaming Cafe', 'Café', 'Movie Theater',
       'Financial or Legal Service', 'Breakfast Spot', 'Burger Joint',
       'Dessert Shop', 'Bar', 'Lounge', 'Juice Bar', 'Multiplex',
       'Bakery', 'Donut Shop', 'Chinese Restaurant', 'Italian Restaurant',
       'Tea Room', 'Bookstore', 'Pizza Place', 'Sandwich Place',
       'Tibetan Restaurant', 'Gym', 'Park', 'Punjabi Restaurant',
       'Brewery', 'Diner', 'Liquor Store', 'Snack Place', 'Restaurant',
       'Paper / Office Supplies Store', 'Coffee Shop',
       'Mexican Restaurant', 'Persian Restaurant', 'Asian Restaurant',
       'Comfort Food Restaurant', 'Ice Cream Shop', 'Indian Sweet Shop',
       'Fast Food Restaurant', 'Kerala Restaurant',
       'South Indian Restaurant', 'Andhra Restaurant',
       'Seafood Restaurant', 'Shopping Mall', 'Thai Restaurant',
       'Middle Eastern Restaurant', 'Buffet', 'Gym Pool',
       'Fried Chicken Joint', 'Bus Station', 'Food Truck'], dtype=object)

## 6. Anlyzing each neighborhood.

In [27]:
# one hot encoding
blr_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
blr_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [blr_onehot.columns[-1]] + list(blr_onehot.columns[:-1])
blr_onehot = blr_onehot[fixed_columns]

print(blr_onehot.shape)
blr_onehot.head(20)

(7523, 244)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Toll Plaza,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category.

In [28]:
blr_grouped = blr_onehot.groupby(["Neighborhoods"]).sum().reset_index()

print(blr_grouped.shape)
blr_grouped.head(20)

(139, 244)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Bengali Restaurant,Big Box Store,Bike Shop,Bistro,Boarding House,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cantonese Restaurant,Capitol Building,Caribbean Restaurant,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cricket Ground,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Financial or Legal Service,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,General Entertainment,German Restaurant,Go Kart Track,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Health Food Store,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hospital,Hotel,Hotel Bar,Hotel Pool,Hyderabadi Restaurant,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Inn,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kebab Restaurant,Kerala Restaurant,Kids Store,Korean Restaurant,Lake,Light Rail Station,Lighthouse,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,North Indian Restaurant,Office,Other Great Outdoors,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Planetarium,Platform,Playground,Plaza,Pool,Pool Hall,Pub,Punjabi Restaurant,Racetrack,Rajasthani Restaurant,Recreation Center,Resort,Rest Area,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shipping Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Tech Startup,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Theater,Tibetan Restaurant,Toll Booth,Toll Plaza,Tourist Information Center,Toy / Game Store,Track Stadium,Trail,Train Station,Travel & Transport,Turkish Restaurant,Udupi Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Adugodi,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,3,0,0,0,1,1,0,0,1,0,1,0,0,0,0,0,0,0,13,0,0,0,0,0,0,4,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,2,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16,1,0,0,0,0,2,0,0,2,0,0,3,0,0,0,0,0,1,4,0,0,0,0,0,2,1,0,0,0,0,0,0,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,3,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,2,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
len((blr_grouped[blr_grouped["Vegetarian / Vegan Restaurant"] > 0]))
#We see there are 57 Vegetarian restaurants in Bangalore and we should select a location where there are less number of such restaurants as 
#it helps in growing business

57

## 7. Create a dataframe for Vegetarian Restaurants data only

In [30]:
blr_veg = blr_grouped[["Neighborhoods","Vegetarian / Vegan Restaurant"]]

In [31]:
blr_veg.head(20)

,Neighborhoods,Vegetarian / Vegan Restaurant
0,Adugodi,0
1,Ananthnagar,0
2,Anjanapura,0
3,Arekere,2
4,Austin Town,0
5,BTM Layout,3
6,Babusapalya,2
7,"Bagalur, Bangalore Urban",1
8,Bahubalinagar,0
9,Baiyyappanahalli,0


## 8. Now cluster the neighbourhoods

Run k-means to cluster the neighborhoods in Bangalore into 3 clusters.

In [32]:
# set number of clusters
kclusters = 3

blr_clustering = blr_veg.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(blr_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


array([0, 0, 0, 1, 0, 1, 1, 2, 0, 0], dtype=int32)

In [33]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
blr_merged = blr_veg.copy()

# add clustering labels
blr_merged["Cluster Labels"] = kmeans.labels_

In [34]:
blr_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
blr_merged.head(10)

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels
0,Adugodi,0,0
1,Ananthnagar,0,0
2,Anjanapura,0,0
3,Arekere,2,1
4,Austin Town,0,0
5,BTM Layout,3,1
6,Babusapalya,2,1
7,"Bagalur, Bangalore Urban",1,2
8,Bahubalinagar,0,0
9,Baiyyappanahalli,0,0


In [35]:
#Add latitude and longitude values by using the join operation
blr_merged = blr_merged.join(df_blr.set_index("Neighborhood"), on="Neighborhood")

print(blr_merged.shape)
blr_merged['Latitude'] = df_coords['Latitude']
blr_merged['Longitude'] = df_coords['Longitude']
blr_merged.head()

(139, 5)


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Adugodi,0,0,12.94401,77.60797
1,Ananthnagar,0,0,12.84292,77.48582
2,Anjanapura,0,0,12.95408,77.54135
3,Arekere,2,1,12.85811,77.55910
4,Austin Town,0,0,12.88567,77.59673


In [36]:
# sorting the results by Cluster Labels
print(blr_merged.shape)
blr_merged.sort_values(["Cluster Labels"], inplace=True)
blr_merged

(139, 5)


,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
0,Adugodi,0,0,12.944010,77.607970
91,Milk Colony,0,0,13.032340,77.558640
90,Mathikere,0,0,13.049640,77.611830
85,"Mahadevapura, Bangalore",0,0,12.920520,77.620900
83,Lingarajapuram,0,0,12.967510,77.715000
80,Krishnarajapuram,0,0,12.920040,77.625460
78,Koppa gate,0,0,12.882290,77.569230
76,"Kodihalli, Bangalore",0,0,13.065830,77.572170
75,Kodigehalli,0,0,12.908680,77.487180
74,Kengeri,0,0,13.014260,77.636740


In [37]:
blr_merged['Vegetarian / Vegan Restaurant'].max()


4

## 9. Visualize the resulting clusters in map

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(blr_merged['Latitude'], blr_merged['Longitude'], blr_merged['Neighborhood'], blr_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 10. Examine each clusters.

#### Cluster 0

In [92]:
blr_merged.loc[blr_merged['Cluster Labels'] == 0]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
69,Kalasipalyam,2,0,12.965530,77.666660
118,"Statue of Edward VII, Bangalore",2,0,12.871560,77.638460
116,"Siddapura, Bangalore",1,0,12.987180,77.603980
112,Sanjaynagar,1,0,13.062710,77.585500
39,Electronic City,1,0,12.946970,77.625590
41,G. M. Palya,1,0,12.998940,77.612760
42,Gandhi Bazaar,1,0,12.975410,77.661550
43,"Gandhi Nagar, Bangalore",1,0,12.944800,77.572120
105,Ramachandrapura,1,0,12.931780,77.526680
46,Garudacharpalya,1,0,13.014450,77.588080


#### Cluster 1

In [93]:
blr_merged.loc[blr_merged['Cluster Labels'] == 1]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
124,Thindlu,0,1,12.922400,77.610270
92,"Murphy Town, Bangalore",0,1,12.966590,77.653180
91,Milk Colony,0,1,13.032340,77.558640
90,Mathikere,0,1,13.049640,77.611830
117,Singasandra,0,1,12.956130,77.731930
134,"Whitefield, Bangalore",0,1,13.037920,77.618630
122,Tannery Road,0,1,12.888130,77.728070
85,"Mahadevapura, Bangalore",0,1,12.920520,77.620900
83,Lingarajapuram,0,1,12.967510,77.715000
81,Kumaraswamy Layout,0,1,13.000390,77.683680


#### Cluster 2

In [94]:
blr_merged.loc[blr_merged['Cluster Labels'] == 2]

,Neighborhood,Vegetarian / Vegan Restaurant,Cluster Labels,Latitude,Longitude
103,Rajajinagar,4,2,13.024450,77.59583
79,Koramangala,3,2,12.800940,77.63019
102,R. T. Nagar,3,2,12.894340,77.58507
5,BTM Layout,3,2,12.963480,77.61297
82,Kundalahalli,3,2,12.898190,77.55927
110,Sadashivanagar,3,2,12.962330,77.60123
99,Palace Guttahalli,3,2,12.918180,77.55925
18,Bilekahalli,3,2,12.900080,77.60432
87,Malleswaram,4,2,13.016350,77.54481
113,Seshadripuram,3,2,13.036940,77.57457


## Final Observation

The cluster analysis shows that the vegan restaurants are most frequent in cluster 2 which suggest that this place won't be much suitable for opening up a new restaurant. Also, when we look at cluster 1 , the numbers won't even say there exists a veg restaurant which could also mean that the business could be low due to lack of customer visits. For a fresh start, being in a too competitive or in no competitive location, a restaurant might see a decline in its business as for highly competitive places, the localities already few brands that attracts customers and in that way a new business would not meet much of a chance to maintain everything on flow. Also, when we look for a non competitive locations, there may start a trend in lacking skill, neatness, hygiene factors because unless there is a little competition, the restaurant quality might get lazy and hence restaurant fails. So, by looking at all these facts, we can say that the best location to opt is from cluster 0 as there are few restaurants with same cuisine and that means there is a good competition that helps him to maintain the quality upto standard and also the customers will gradually happen to give a visit. 